![title](https://acreditta.b-cdn.net/wp-content/uploads/2025/07/logotipo-Tecmilenio-grande.webp)

# Pandas

## 1. Objetivo

Procesar en Python la información de Oilst de forma funcional para el análisis de los retrasos en las órdenes de los clientes.

## Datos de Oilst

Datos disponibles:

| Archivo                      	| Descripción                                                                                                                                     	| Formato 	|
|------------------------------	|-------------------------------------------------------------------------------------------------------------------------------------------------	|---------	|
| olist_customers_dataset      	| Datos de identificación de los clientes y la ubicación del domicilio que tienen registrado con Olist para recibir pedidos.                      	| xlsx    	|
| olist_orders_dataset         	| Datos de identificación de las órdenes realizadas por los clientes, su estatus de envío y la cronología de entrega en el domicilio del cliente. 	| csv     	|
| olist_geolocation_dataset    	| Datos de geolocalización de códigos postales de zonas de Brasil.                                                                                	| csv     	|
| olist_order_items_dataset    	| Contiene la relación de artículos contenidos en las órdenes de los clientes                                                                     	| csv     	|
| olist_order_payments_dataset 	| Contiene la relación de pagos que cada cliente hizo en sus órdenes por medio de pago                                                            	| csv     	|
| states_abreviations          	| Contiene la relación de nombres de los estados de Brasil y sus abreviaciones.                                                                   	| json    	|


Adicionalmente, también se tiene el diagrama de relación entre las tablas:

![title](oilst_diagram.png)



Este documento se desarrollarán scripts en Python que permitan procesar la  la información de Oilst para realizar posteriormente el análisis de sus datos

## 2. Librerias de trabajo

In [1]:
import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)


## 3. Lectura de datos

Primero nos encargaremos de leer los datos, indicando a Python donde se encuentra la carpeta que contiene los datos y los nombres de los archivos relevantes para el análisis.

In [2]:
#  Indicamos la ruta a la carpeta de de tu computadora 
DATA_PATH="/Users/cesar/sandbox/data_analysis_python/data/olist/"

Ahora procederemos a definir variables que indiquen el nombre de los archivos junto con su extensión (por ejemplo, `.csv`, `.json` u otra).

In [3]:
# pip install sqlalchemy psycopg2-binary

In [4]:
!pwd

/Users/cesar/sandbox/data_analysis_python/notebooks


In [5]:
!ls -lha1 /Users/cesar/sandbox/data_analysis_python/data/oilst/

ls: /Users/cesar/sandbox/data_analysis_python/data/oilst/: No such file or directory


In [6]:
FILE_CUSTOMERS = 'olist_customers_dataset.xlsx'
FILE_GEOLOCATIONS = 'olist_geolocation_dataset.csv'
FILE_ITEMS = 'olist_order_items_dataset.csv'
FILE_ORDERS = 'olist_orders_dataset.csv'
FILE_STATES_ABBREVIATIONS = 'states_abbreviations.json'

Echaremos mano de la utilidad `os.path.join` de Python que indicar rutas en tu computadora donde se ubican archivos, así Pandas encontrá los archivos de datos.


**Ejemplo**

A continuación mostraremos un ejemplo leyendo el archivo `olist_geolocation_dataset.csv`:

In [7]:
# Ejemplo
print(f"Ruta del archivo: {FILE_GEOLOCATIONS}")
print(os.path.join(DATA_PATH, FILE_GEOLOCATIONS))

Ruta del archivo: olist_geolocation_dataset.csv
/Users/cesar/sandbox/data_analysis_python/data/olist/olist_geolocation_dataset.csv


In [8]:
# Leemos con pandas
geolocations = pd.read_csv(
    os.path.join(DATA_PATH, FILE_GEOLOCATIONS),
    dtype={'geolocation_zip_code_prefix': 'str'}
    )

Podemos explorar el archivo con los comandos `.head(), .info(), .describe()`

**Función .head()**

Este comando nos brinda información de los primeros renglones de la tabla:

In [9]:
geolocations.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,01037,-23.545621,-46.639292,sao paulo,SP
1,01046,-23.546081,-46.644820,sao paulo,SP
2,01046,-23.546129,-46.642951,sao paulo,SP
3,01041,-23.544392,-46.639499,sao paulo,SP
4,01035,-23.541578,-46.641607,sao paulo,SP


**Función .describe()**

Este comando nos ayuda obtener información estadística de las variables de la tabla, entre ellas el conteo de los elementos no nulos de una columnas, la media, su desviación estándar (que es una medida de que tan dispersa se encuentra la información a partir de su media), el primer, segundo y tercer cuartil de los datos 
que se refiere a que valor de los datos ocupar el 25%, 50% y 75% de la distribución de los datos si se ordenan de menor a mayor y que se denotan, respectivamente como, denotado Q1, Q2 y Q3. Además provee información del mínimo y máximo presentes en la columna.

In [10]:
geolocations .describe()

,geolocation_lat,geolocation_lng
count,1.000163e+06,1.000163e+06
mean,-2.117615e+01,-4.639054e+01
std,5.715866e+00,4.269748e+00
min,-3.660537e+01,-1.014668e+02
25%,-2.360355e+01,-4.857317e+01
50%,-2.291938e+01,-4.663788e+01
75%,-1.997962e+01,-4.376771e+01
max,4.506593e+01,1.211054e+02


**Función .info()**

Este comando nos ayuda obtener información de cuantas entradas tiene una tabla, el tipo de dato en que se representarn sus columnas y el tamaño que ocupa en la memoria de nuestra computadora.

In [11]:
geolocations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  object 
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), object(3)
memory usage: 38.2+ MB


Ahora estamos en posición de completar el código para el resto de conjuntos de datos:

### 3.1 Archivo olist_customers_dataset

**Hint:** 

1. Pandas necesita instalar una libreria extrar para leer éste tipo de archivos (`openpyxl`).
2. Se debe expecificar el tipo de dato de `customer_zip_code_prefix` (ver Anexo A)

In [12]:
customers = pd.read_excel(
    os.path.join(DATA_PATH, FILE_CUSTOMERS),
    dtype={'customer_zip_code_prefix': str}
    )

Verificamos que `customer_zip_code_prefix` tenga el formato correcto (!no es un número, sino un código postal!)

In [13]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  object
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: object(5)
memory usage: 3.8+ MB


También podemos visualizar una muestra aleatoria con el método `.sample(numero_entero)`

In [14]:
# Revisamos 10 renglones al azar
customers.sample(10)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
79132,3db8f291923971467c299f44c051a6e9,1b2151cb7df260047594c28f94a1e1d6,69054,manaus,AM
69697,bf84db1217d83f8ac1838d375b7784f3,74f6957797ebda73b6c3990e98cc0ad5,18021,sorocaba,SP
33475,20beaf3319dcb7deec6ab4db1e559e8d,f00feb2d02bbc58957ea2cd1d9b4f815,13272,valinhos,SP
65380,bee5ac61e1c115db72e7c6843a3d1932,e60c3e8f3984efc7f5107d6aacc98da2,07081,guarulhos,SP
87284,245bf42163498d23ca14484c07988535,9dab217089224ad75d6f756a5b253930,22280,rio de janeiro,RJ
20756,f7b249c9618f84a66a70a8c127d06105,954456f376f79dfa36460750500f7dc6,88015,florianopolis,SC
54988,f0dc368b018c4102a459934a02118a1c,b767ddfe8c6a388587c3abb6ec4b464a,35790,curvelo,MG
55056,11259004e69dac0f64c28698b2e702d3,4ed16134a4f12d79e401be023468cbe1,24936,marica,RJ
88733,817b266d9c8e7ef78fe7a9c3f4faacbf,d045f088e61a2992328a462929fcfc9a,52170,recife,PE
61272,6ac4f4b8a8559141e69a781be84aacee,9d7b7634f5b8ca6256a702c210021d28,17501,marilia,SP


### 3.2 Archivo olist_order_items_dataset

In [15]:
items = pd.read_csv(
    os.path.join(DATA_PATH, FILE_ITEMS),
    )

Como sabemos, este conjunto contiene datos de los productos que contiene cada orden. Por ello, para el análisis nos interesará saber cual es la cantidad de productos en cada orden y el precio total de las mismas.

Esto se puede calcular mediante agregaciones de Pandas (https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.core.groupby.DataFrameGroupBy.agg.html), que basicamente nos permite hacer cálculos para un grupo en especial. En el ejemplo de inferior se muestra para cada `order_id` se cuenta la cantidad de productos (items) y el precio agregado de todos los artículos en las órdenes:

In [16]:
items_agg = items.groupby(
    ['order_id']).agg(
        # conteo de producto
        {'order_item_id': 'count',
        # suma de los precios de los artículos
        'price': 'sum'}
                      ).reset_index() 

In [17]:
items_agg.head()

,order_id,order_item_id,price
0,00010242fe8c5a6d1ba2dd792cb16214,1,58.90
1,00018f77f2f0320c557190d7a144bdd3,1,239.90
2,000229ec398224ef6ca0657da4fc703e,1,199.00
3,00024acbcdf0a6daa1e931b038114c75,1,12.99
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,199.90


Vamos a renombrar las columnas anteriores, para que sea más intuitivo su significado.

In [18]:
# Nota: el parámetro inplace sobre escribe los cambios
# en el dataframe
items_agg.rename(
    columns={'order_item_id': 'total_products', 'price': 'total_sales'},
    inplace=True
    )

In [19]:
items_agg

,order_id,total_products,total_sales
0,00010242fe8c5a6d1ba2dd792cb16214,1,58.90
1,00018f77f2f0320c557190d7a144bdd3,1,239.90
2,000229ec398224ef6ca0657da4fc703e,1,199.00
3,00024acbcdf0a6daa1e931b038114c75,1,12.99
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,199.90
...,...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,1,299.99
98662,fffcd46ef2263f404302a634eb57f7eb,1,350.00
98663,fffce4705a9662cd70adb13d4a31832d,1,99.90
98664,fffe18544ffabc95dfada21779c9644f,1,55.99


Vamos a repetir el proceso anterior para el resto de tablas:

### 3.3 olist_order_payments_dataset

*Tenemos que saca los datos de una base SQL!!*

In [20]:
from sqlalchemy import create_engine

DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "tecmilenio"
DB_USER = "postgres"
DB_PASSWORD = "postgres"

# --- 2. Define the SQL Query ---
sql_query = """
SELECT
    order_id,
    payment_sequential,
    payment_type,
    payment_installments,
    payment_value
FROM tecmilenio.public.olist_order_payments_dataset;
"""

engine_url = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(engine_url)
payments = pd.read_sql(sql_query, engine)

In [44]:
payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


### 3.4 states_abbreviations

In [21]:
states_abbreviations = pd.read_json(
    os.path.join(DATA_PATH, FILE_STATES_ABBREVIATIONS)
    )

### 3.5 olist_orders_dataset

In [22]:
orders = pd.read_csv(
    os.path.join(DATA_PATH, FILE_ORDERS)
    )

Si revisamos el formato que Pandas ha otorgado a las columnas que contienen fecha encontraremos algo extraño, no se ha interpretado correctamente:

In [23]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       99441 non-null  object 
 1   customer_id                    99441 non-null  object 
 2   order_status                   99441 non-null  object 
 3   order_purchase_timestamp       99441 non-null  object 
 4   order_approved_at              99281 non-null  object 
 5   order_delivered_carrier_date   97658 non-null  object 
 6   order_delivered_customer_date  96476 non-null  object 
 7   order_estimated_delivery_date  99441 non-null  object 
 8   distance_distribution_center   96470 non-null  float64
dtypes: float64(1), object(8)
memory usage: 6.8+ MB


Por lo tanto es necesario convertir las columnas al formato de fecha `datetime` (Ver: https://towardsdatascience.com/working-with-datetime-in-pandas-dataframe-663f7af6c587).

Emplearemos la función `to_datetime` de Pandas para modificar las columnas para que se interpreten como fechas:

In [24]:
# Conveirte a formato fecha
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'], errors='coerce' )
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'], errors='coerce')
orders['order_delivered_carrier_date'] = pd.to_datetime(orders['order_delivered_carrier_date'], errors='coerce')
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'], errors='coerce')
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'], errors='coerce')

In [25]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
 8   distance_distribution_center   96470 non-null  float64       
dtypes: datetime64[ns](5), float64(1), object(3)
memory usage: 6.8+ MB


Una vez corregido el tema de las fecha definiremos también algunas variables auxiliares que nos serviran posteriormente en el análisis.

Por ejemplo, es de interés conocer en que mes, año y trimestre del año sucedieron las compras, usando la columna `order_purchase_timestamp`:

In [26]:
# Define una columna con el año en que sucedió la orden
orders['year'] = orders['order_purchase_timestamp'].dt.year

# Define una columna con el mes en que sucedió la orden
orders['month'] = orders['order_purchase_timestamp'].dt.month

# Define una columna con trimestre con el que paso la orden (ej. Q12018)
orders['quarter'] = orders['order_purchase_timestamp'].dt.to_period('Q')

# Define una columna con mes y año con el que paso la orden (ej. 02-2018)
orders['year_month'] = orders['order_purchase_timestamp'].dt.to_period('M')

Por otro lado, también necesitamos identificar las órdenes que tuvieron retrasos prolongados. Recordemos que de acuerdo a la documentación del `Anexo A`:
* Oilst notifica el usuario de cuando llegará su pedido con el valor de la columna `order_estimated_delivery_date`,
* Además la fecha real en que se llevó la entrega se encuentra en el campo `order_delivered_customer_date`

A continuación calcularemos distancia (en días) entre ambas fecha definiendo a la variable `delta_days`:

In [27]:
# Nota: tenemos que realizar la conversion de
# segundos a días

orders['delta_days'] = (
    orders['order_delivered_customer_date'] -
    orders['order_estimated_delivery_date']
    ).dt.total_seconds()/ 60 / 60 / 24

**Pregunta:**

* ¿Porqué para convertir lo anterior a días tenemos que dividir entre 60, después entre 60 y luego entre 24?

Podemos explorar ahora los valores de dicha variables con el método `.descri be()`

In [28]:
orders['delta_days'].describe()

count    96476.000000
mean       -11.179120
std         10.186113
min       -146.016123
25%        -16.244384
50%        -11.948941
75%         -6.390000
max        188.975081
Name: delta_days, dtype: float64

En el contexto del problema, los valores de `delta_days` tiene el significado:

* Un valor negativo en `delta_days` significa que el pedido llego antes de lo esperado; es decir, no existió retraso.
* Un valor de `delta_days`, mayor a 0 días pero menor a 3 días, significa que es un retrazo aceptable,
* Sin embargo, si `delta_days` es más grande que 3 días esto significa que tenemos un retrazo prolongado.

Crearemos una variable `delay_status` para indicar la discusión anterior usando el operador `where` de Numpy (https://towardsdatascience.com/creating-conditional-columns-on-pandas-with-numpy-select-and-where-methods-8ee6e2dbd5d5).

Esencialmente, el operador `where` de Numpy permite definir variables siguiendo reglas lógicas de manera condicional, similar al `if ... else ...` de Python:

In [29]:
# Define 
orders['delay_status']  = np.where(
    orders['delta_days'] > 3, 'long_delay',
    np.where(orders['delta_days'] <= 0, 'on_time','short_delay')
    )

Para ver el efecto de lo anterior podemos extraer un muestra con la función `.sample`

In [30]:
orders['delay_status'].sample(10)

22091        on_time
40888        on_time
50297        on_time
73929        on_time
3786     short_delay
80964    short_delay
98066        on_time
75290        on_time
64145        on_time
98396        on_time
Name: delay_status, dtype: object

**Pregunta:**

* ¿Las categorías anteriores en que se han clasificado los estatus de entrega de las órdenes se pueden inteserctar?


### 3.5 olist_geolocation_dataset

Aunque anteriormente hemos leído este archivo, debemos notar que contiene información redudante de muchos codigos postales, como en el caso del valor `24220`:


In [31]:
geolocations.query("geolocation_zip_code_prefix == 24220")

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state


Para el análisis tendremos que eliminar esta duplicaciones. Esto se puede lograr con el método `drop_duplicates`

In [32]:
unique_geolocations = geolocations.drop_duplicates(
    subset = ['geolocation_zip_code_prefix']
    )

Como se aprecia a continuación, ahora el dataframe `unique_geolocations` corrige el error:

In [33]:
unique_geolocations.query(
    "geolocation_zip_code_prefix == 24220"
    )

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state


## 4. Procesamiento global

Ahora que hemos cargado a Pandas los datos del E-commerce, debemos **consolidar toda la información** en una sola tabla, lo que nos permitirá centralizar el análisis y hacer comparativos.

Para ello, nos proponemos lo siguiente:
1. A los datos de clientes le añadiremos los datos de geolocalización. **(Clientes + geolocalización)**
2. Tales datos se complementarán añadiendo los datos del nombre del estado de Brasil en que se localizan. (**Clientes + geolocalización + nombre del estado donde viven**)
3. Posteriormente archivo de órdenes, agregaremos los datos del precio y cantidad de artículos. **(Órdenes + total de artículos y precios)**
4. Finalmente, uniremos toda la información de los pasos 2 y 3 en una sola tabla.

### 4.1 Clientes + geolocalización

Para unir dos fuentes de datos, podemos usar la función `.merge` (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html). En el siguiente ejemplo se unes los datos de los clientes junto sus geolocalizaciones.

**Nota:** Los códigos postales deben tener el formato texto.

In [34]:
customers_geolocation = customers.merge(
    unique_geolocations,
    left_on='customer_zip_code_prefix',
    right_on='geolocation_zip_code_prefix',
    how='left'
)

In [35]:
customers_geolocation.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,14409,-20.509897,-47.397866,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,09790,sao bernardo do campo,SP,09790,-23.726853,-46.545746,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,01151,sao paulo,SP,01151,-23.527788,-46.660310,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,08775,mogi das cruzes,SP,08775,-23.496930,-46.185352,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,13056,-22.987222,-47.151073,campinas,SP


### 4.2 Clientes + geolocalización + nombre del estado donde viven

Ahora repetiremos un proceso análogo pero con los nombres del estado donde viven los customers

In [36]:
customers_geolocation_estado = customers_geolocation.merge(
    states_abbreviations,
    left_on='geolocation_state',
    right_on='abbreviation',
    how='left'
)

In [37]:
customers_geolocation_estado

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,abbreviation,state_name
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,14409,-20.509897,-47.397866,franca,SP,SP,São Paulo
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,09790,sao bernardo do campo,SP,09790,-23.726853,-46.545746,sao bernardo do campo,SP,SP,São Paulo
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,01151,sao paulo,SP,01151,-23.527788,-46.660310,sao paulo,SP,SP,São Paulo
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,08775,mogi das cruzes,SP,08775,-23.496930,-46.185352,mogi das cruzes,SP,SP,São Paulo
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,13056,-22.987222,-47.151073,campinas,SP,SP,São Paulo
...,...,...,...,...,...,...,...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,03937,sao paulo,SP,03937,-23.587901,-46.501830,são paulo,SP,SP,São Paulo
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,06764,taboao da serra,SP,06764,-23.612294,-46.765787,taboao da serra,SP,SP,São Paulo
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE,60115,-3.744128,-38.510859,fortaleza,CE,CE,Ceará
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS,92120,-29.956391,-51.167614,canoas,RS,RS,Rio Grande do Sul


### 4.3 Órdenes + total de artículos y precios

In [38]:
orders_totals = orders.merge(
    items_agg,
    on=['order_id'],
    how='left'
    )

In [39]:
orders_totals

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,distance_distribution_center,year,month,quarter,year_month,delta_days,delay_status,total_products,total_sales
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,29.84,2017,10,2017Q4,2017-10,-7.107488,on_time,1.0,29.99
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,25.53,2018,7,2018Q3,2018-07,-5.355729,on_time,1.0,118.70
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,61.56,2018,8,2018Q3,2018-08,-17.245498,on_time,1.0,159.90
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,60.30,2017,11,2017Q4,2017-11,-12.980069,on_time,1.0,45.00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,80.97,2018,2,2018Q1,2018-02,-9.238171,on_time,1.0,19.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28,68.26,2017,3,2017Q1,2017-03,-10.369433,on_time,1.0,72.00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,14.92,2018,2,2018Q1,2018-02,-1.265324,on_time,1.0,174.90
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,36.37,2017,8,2017Q3,2017-08,-5.524803,on_time,1.0,205.99
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,32.04,2018,1,2018Q1,2018-01,-20.018819,on_time,2.0,359.98


### 4.4 Clientes + geolocalización + nombre del estado donde viven + Órdenes + total de artículos y precios

In [40]:
results = orders_totals.merge(
    customers_geolocation_estado,
    on=['customer_id'],
    how='left'
    )

In [41]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
 8   distance_distribution_center   96470 non-null  float64       
 9   year                           99441 non-null  int32         
 10  month                          99441 non-null  int32         
 11  quarter        

In [42]:
results.sample(5)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,distance_distribution_center,year,month,quarter,year_month,delta_days,delay_status,total_products,total_sales,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,abbreviation,state_name
56121,3048257a09e97a57f58f5d0586cc01a3,b995f956b15b59222e2a3ad9db72d562,delivered,2018-01-25 16:24:07,2018-01-25 16:34:50,2018-01-29 20:04:19,2018-02-06 23:23:52,2018-02-20,79.41,2018,1,2018Q1,2018-01,-13.025093,on_time,1.0,24.99,429de3d8004cbe46652a0e069fc545e3,21073,rio de janeiro,RJ,21073,-22.848618,-43.269667,rio de janeiro,RJ,RJ,Rio de Janeiro
51546,172ce73720737cdb83fcfeca26ed4d70,f3799a6efc9956211770b7f5554a886e,delivered,2018-05-20 11:48:50,2018-05-22 08:34:06,2018-05-22 13:31:00,2018-05-24 19:27:33,2018-06-05,88.04,2018,5,2018Q2,2018-05,-11.189201,on_time,1.0,157.89,73f6c63a7d887f8f77dbb92fb0bee026,13890,aguas da prata,SP,13890,-21.936065,-46.716361,aguas da prata,SP,SP,São Paulo
57606,1733916305727c04da99e1ceab781c93,5747e92c2d001d68a632852f8a4f3768,delivered,2017-05-10 14:08:58,2017-05-10 14:22:33,2017-05-15 08:55:50,2017-05-18 11:08:01,2017-05-29,31.06,2017,5,2017Q2,2017-05,-10.536100,on_time,1.0,30.00,9f16fd7755d607cff890d9af2f212340,06708,cotia,SP,06708,-23.585162,-46.841815,cotia,SP,SP,São Paulo
85037,17e3bdfb40f275e758587eb0df1e7516,d3635b8eeab30334c8add14ca042370e,delivered,2018-03-28 16:56:28,2018-03-28 17:08:05,2018-04-02 20:10:39,2018-04-03 23:05:35,2018-04-10,13.57,2018,3,2018Q1,2018-03,-6.037789,on_time,1.0,25.00,9b72a613e97d3ae0a93eb1af7d57b4f1,01050,sao paulo,SP,01050,-23.549774,-46.645599,sao paulo,SP,SP,São Paulo
89464,a167a25e5e6faf3098a732b7a75a41d5,ee1efc7f165c3d3c8dc2325d82e79b2d,delivered,2018-03-04 21:25:17,2018-03-06 03:55:59,2018-03-06 18:16:56,2018-03-15 20:50:42,2018-04-03,71.22,2018,3,2018Q1,2018-03,-18.131458,on_time,1.0,89.90,791fe8994300d9034bf3bb34f2376e56,99670,ronda alta,RS,99670,-27.783656,-52.807832,ronda alta,RS,RS,Rio Grande do Sul


In [43]:
results.to_csv("oilst_processed.csv", index=False)